In [1]:
import os
import itertools
import pandas as pd
import numpy as np
from darts import TimeSeries
from darts.models import TSMixerModel
from darts.dataprocessing.transformers import Scaler, Diff
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
import warnings
import traceback

warnings.filterwarnings("ignore")

# Suppress PyTorch Lightning logs
import pytorch_lightning as pl
pl.utilities.rank_zero_info = lambda *args, **kwargs: None
pl.utilities.rank_zero_warn = lambda *args, **kwargs: None
pl.utilities.rank_zero_debug = lambda *args, **kwargs: None

# ===================================================================
# CONFIGURATION
# ===================================================================

DATASETS = {
    "IHSG": r"C:\wilson\Thesis\Data\final_data\ihsg_final.csv",
    "LQ45": r"C:\wilson\Thesis\Data\final_data\lq45_final.csv",
    "Kompas100": r"C:\wilson\Thesis\Data\final_data\kompas100_final.csv"
}

SCENARIOS = {
    "Short-Term Sprint": {"window": 30, "horizon": 1},
    "Medium-Term Pace": {"window": 60, "horizon": 5},
    "Long-Term Marathon": {"window": 90, "horizon": 10},
}

COVARIATE_SETS = {
    "None": [],
    "Domestic": ["close_inflation", "close_gdp", "close_cpi", "close_unemployment"],
    "Global": ["close_usd", "close_brent", "close_wti"],
    "All": ["close_inflation", "close_gdp", "close_cpi", "close_unemployment",
            "close_usd", "close_brent", "close_wti"],
    "Inflation": ["close_inflation"],
    "GDP": ["close_gdp"],
    "CPI": ["close_cpi"],
    "Unemployment": ["close_unemployment"],
    "USD": ["close_usd"],
    "Brent": ["close_brent"],
    "WTI": ["close_wti"]
}

# Fixed TSMixer hyperparameters (Matches Felix & Gunawan 2020)
TSMIXER_CONFIG = {
    "dropout": 0.4,
    "hidden_size": 64,
    "batch_size": 64,
    "learning_rate": 0.001,
    "n_epochs": 100,
    "num_blocks": 4
}

RESULTS_PATH = r"C:\wilson\Thesis\Results\tsmixer_results_logret.csv"
PREDICTION_DIR = r"C:\wilson\Thesis\Results\tsmixer_predictions_logret"
os.makedirs(os.path.dirname(RESULTS_PATH), exist_ok=True)
os.makedirs(PREDICTION_DIR, exist_ok=True)

# ===================================================================
# HELPER FUNCTION (UPDATED WITH LOG-RETURN PIPELINE)
# ===================================================================

def train_and_evaluate_tsmixer(df, target_col, covariate_cols, input_chunk_length, output_chunk_length,
                               test_ratio=0.2):
    try:
        # 1. Sort & prepare target
        df = df.sort_values("date").reset_index(drop=True)

        target_series = TimeSeries.from_dataframe(df, time_col="date", value_cols=target_col,
                                                  fill_missing_dates=True, freq="B")
        target_series = TimeSeries.from_dataframe(
            target_series.to_dataframe().fillna(method="ffill").fillna(method="bfill")
        )

        # 2. Prepare Covariates
        covariate_series = None
        if covariate_cols:
            covariate_series = TimeSeries.from_dataframe(df, time_col="date", value_cols=covariate_cols,
                                                         fill_missing_dates=True, freq="B")
            covariate_series = TimeSeries.from_dataframe(
                covariate_series.to_dataframe().fillna(method="ffill").fillna(method="bfill")
            )

        # 3. TRANSFORM: Log -> Diff (Log Returns)
        # This matches the TFT and RF pipeline exactly
        ts_log = target_series.map(np.log)
        differencer = Diff(lags=1)
        ts_log_diff = differencer.fit_transform(ts_log)

        # 4. Train/test split (On the transformed data)
        train_cutoff = int(len(ts_log_diff) * (1 - test_ratio))
        target_train, target_test = ts_log_diff[:train_cutoff], ts_log_diff[train_cutoff:]
        
        cov_train, cov_test = None, None
        if covariate_series:
            # Align covariates with the shifted (differenced) target
            cov_aligned = covariate_series.slice_intersect(ts_log_diff)
            cov_train, cov_test = cov_aligned[:train_cutoff], cov_aligned[train_cutoff:]

        # 5. Scaling (MinMax on Returns)
        scaler_y = Scaler()
        target_train_scaled = scaler_y.fit_transform(target_train)
        target_test_scaled = scaler_y.transform(target_test)

        cov_train_scaled, cov_test_scaled = None, None
        if covariate_series:
            scaler_x = Scaler()
            cov_train_scaled = scaler_x.fit_transform(cov_train)
            cov_test_scaled = scaler_x.transform(cov_test)

        # 6. Train TSMixer
        model = TSMixerModel(
            input_chunk_length=input_chunk_length,
            output_chunk_length=output_chunk_length,
            dropout=TSMIXER_CONFIG["dropout"],
            hidden_size=TSMIXER_CONFIG["hidden_size"],
            use_reversible_instance_norm=True,
            n_epochs=TSMIXER_CONFIG["n_epochs"],
            batch_size=TSMIXER_CONFIG["batch_size"],
            random_state=42,
            optimizer_kwargs={"lr": TSMIXER_CONFIG["learning_rate"]},
            pl_trainer_kwargs={
                "accelerator": "cpu",
                "enable_progress_bar": False,
                "enable_model_summary": False,
                "logger": False
            }
        )

        model.fit(target_train_scaled, past_covariates=cov_train_scaled, verbose=False)

        # 7. Historical Forecasts (Rolling Window)
        full_target_scaled = target_train_scaled.append(target_test_scaled)
        full_cov_scaled = None
        if covariate_series:
            full_cov_scaled = cov_train_scaled.append(cov_test_scaled)

        preds_scaled_ts = model.historical_forecasts(
            series=full_target_scaled,
            past_covariates=full_cov_scaled,
            start=train_cutoff,
            forecast_horizon=output_chunk_length,
            stride=output_chunk_length, # STRIDE = HORIZON for non-overlapping blocks (faster/cleaner)
            retrain=False,
            verbose=False,
            last_points_only=False
        )

        if isinstance(preds_scaled_ts, TimeSeries):
            preds_scaled_ts = [preds_scaled_ts]

        # 8. Reconstruction Loop (Inverse Scale -> Cumulative Sum -> Exp)
        all_pred_values = []
        all_pred_dates = []

        for chunk in preds_scaled_ts:
            # A. Inverse Scale (Get Log Returns)
            chunk_diff_log = scaler_y.inverse_transform(chunk)
            chunk_dates = chunk_diff_log.time_index
            chunk_log_returns = chunk_diff_log.values().flatten()

            # B. Find Anchor Price (Real Log-Price from yesterday)
            start_date = chunk_dates[0]
            # Get index in the LOG series (before differencing)
            try:
                # We need the value BEFORE the start_date to anchor the cumsum
                idx = ts_log.get_index_at_point(start_date)
                anchor_log_price = ts_log[idx-1].values()[0][0]
            except:
                # Fallback if boundary issue (rare)
                continue

            # C. Cumulative Sum to reconstruct Log Prices
            chunk_log_prices = anchor_log_price + np.cumsum(chunk_log_returns)
            
            # D. Exp to get Real Prices
            chunk_prices = np.exp(chunk_log_prices)

            all_pred_values.extend(chunk_prices)
            all_pred_dates.extend(chunk_dates)

        # 9. Build Prediction DataFrame
        prediction_df = pd.DataFrame({
            "date": pd.to_datetime(all_pred_dates),
            "predicted": all_pred_values
        })

        # Merge with Actuals
        full_df = df[["date", target_col]].copy()
        full_df.columns = ["date", "actual"]
        
        final_df = pd.merge(full_df, prediction_df, on="date", how="left")
        
        # Add train_flag
        # Calculate roughly where training ended
        train_end_date = target_train.end_time()
        final_df["train_flag"] = 0
        final_df.loc[final_df["date"] <= train_end_date, "train_flag"] = 1

        # 10. Metrics
        eval_df = final_df.dropna(subset=["predicted"])
        
        if len(eval_df) == 0:
            return {"mse": np.inf, "mae": np.inf, "mape": np.inf, "success": False, "prediction_df": None}

        y_true = eval_df["actual"].values
        y_pred = eval_df["predicted"].values

        mse = np.mean((y_true - y_pred) ** 2)
        mae = np.mean(np.abs(y_true - y_pred))
        mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

        return {
            "mse": mse,
            "mae": mae,
            "mape": mape,
            "success": True,
            "prediction_df": final_df
        }

    except Exception as e:
        traceback.print_exc() 
        print(f"❌ Error during training: {e}")
        return {"mse": np.inf, "mae": np.inf, "mape": np.inf, "success": False, "prediction_df": None}

# ===================================================================
# MAIN EXECUTION LOOP
# ===================================================================

results = []
total_runs = len(DATASETS) * len(SCENARIOS) * len(COVARIATE_SETS)
run_count = 0

print("=" * 90)
print("🚀 STARTING TSMIXER LOG-RETURN EXPERIMENTS (APPLE-TO-APPLE)")
print("=" * 90)
print(f"Total experiments: {total_runs}\n")

for target_name, dataset_path in DATASETS.items():
    df = pd.read_csv(dataset_path)
    df["date"] = pd.to_datetime(df["date"])
    print(f"\n📂 Loaded {target_name} dataset ({len(df)} rows)")

    for scenario_name, scenario_params in SCENARIOS.items():
        input_len = scenario_params["window"]
        output_len = scenario_params["horizon"]

        for cov_name, covariates in COVARIATE_SETS.items():
            run_count += 1
            print(f"[{run_count}/{total_runs}] ▶ Target={target_name} | Scenario={scenario_name} | Covariates={cov_name}")

            cols = ["date", f"close_{target_name.lower()}"]
            if covariates:
                cols += covariates
            scenario_df = df[cols].dropna().copy()

            metrics = train_and_evaluate_tsmixer(
                scenario_df,
                target_col=f"close_{target_name.lower()}",
                covariate_cols=covariates,
                input_chunk_length=input_len,
                output_chunk_length=output_len
            )

            # Save prediction CSV
            if metrics["success"]:
                safe_target = target_name.replace(" ", "_")
                safe_scenario = scenario_name.replace(" ", "_")
                safe_cov = cov_name.replace(" ", "_")

                pred_filename = f"tsmixer_pred_{safe_target}_{safe_scenario}_{safe_cov}.csv"
                pred_path = os.path.join(PREDICTION_DIR, pred_filename)

                prediction_df = metrics["prediction_df"]
                prediction_df["target"] = target_name
                prediction_df["scenario"] = scenario_name
                prediction_df["covariate_set"] = cov_name

                prediction_df.to_csv(pred_path, index=False)

            # Append results summary
            results.append({
                "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "target": target_name,
                "scenario": scenario_name,
                "window": input_len,
                "horizon": output_len,
                "covariate_set": cov_name,
                "mse": metrics["mse"],
                "mae": metrics["mae"],
                "mape": metrics["mape"],
                "success": metrics["success"],
                "dropout": TSMIXER_CONFIG["dropout"],
                "hidden_size": TSMIXER_CONFIG["hidden_size"],
                "batch_size": TSMIXER_CONFIG["batch_size"],
                "learning_rate": TSMIXER_CONFIG["learning_rate"],
                "n_epochs": TSMIXER_CONFIG["n_epochs"]
            })

            # Print results
            if metrics["success"]:
                print(f"   ✅ MAPE={metrics['mape']:.2f}% | MAE={metrics['mae']:.2f}")
            else:
                print("   ❌ FAILED")

            # Save intermediate summary results every 5 runs
            if run_count % 5 == 0:
                pd.DataFrame(results).to_csv(RESULTS_PATH, index=False)

# Save final summary results
results_df = pd.DataFrame(results)
results_df.to_csv(RESULTS_PATH, index=False)

print("\n" + "=" * 90)
print(f"✅ ALL EXPERIMENTS COMPLETE!\nSummary Results:\n{RESULTS_PATH}")
print(f"Prediction CSVs saved to:\n{PREDICTION_DIR}")
print("=" * 90)

C:\Users\ASUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🚀 STARTING TSMIXER LOG-RETURN EXPERIMENTS (APPLE-TO-APPLE)
Total experiments: 99


📂 Loaded IHSG dataset (3764 rows)
[1/99] ▶ Target=IHSG | Scenario=Short-Term Sprint | Covariates=None


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer.fit` stopped: `max_epochs=100` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


   ✅ MAPE=0.57% | MAE=39.92
[2/99] ▶ Target=IHSG | Scenario=Short-Term Sprint | Covariates=Domestic


`Trainer.fit` stopped: `max_epochs=100` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


...

   ✅ MAPE=1.57% | MAE=18.44
[98/99] ▶ Target=Kompas100 | Scenario=Long-Term Marathon | Covariates=Brent


`Trainer.fit` stopped: `max_epochs=100` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


   ✅ MAPE=1.57% | MAE=18.42
[99/99] ▶ Target=Kompas100 | Scenario=Long-Term Marathon | Covariates=WTI


`Trainer.fit` stopped: `max_epochs=100` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


   ✅ MAPE=1.51% | MAE=17.75

✅ ALL EXPERIMENTS COMPLETE!
Summary Results:
C:\wilson\Thesis\Results\tsmixer_results_logret.csv
Prediction CSVs saved to:
C:\wilson\Thesis\Results\tsmixer_predictions_logret
